In [6]:
import os
import re
import json
import logging
import requests
import pandas as pd
from dotenv import load_dotenv
from requests.exceptions import HTTPError

# Configurations
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
load_dotenv()

# Environment variables
client_id = os.getenv("BIGCOMMERCE_CLIENT_ID")
access_token = os.getenv("BIGCOMMERCE_ACCESS_TOKEN")
store_hash = os.getenv("BIGCOMMERCE_STORE_HASH")
BASE_URL_TEMPLATE = f"https://api.bigcommerce.com/stores/{store_hash}/v3/"

class BigCommerceAPI:
    def __init__(self, client_id, access_token, store_hash):
        self.headers = {
            "X-Auth-Client": client_id,
            "X-Auth-Token": access_token,
            "Content-Type": "application/json"
        }
        self.base_url = BASE_URL_TEMPLATE.format(store_hash=store_hash)

    def get_request(self, url):
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except HTTPError as http_err:
            logging.error(f"HTTP error occurred: {http_err}, Response: {response.text}")
        except Exception as err:
            logging.error(f"An error occurred: {err}")
            return None

    # [Add post_request, put_request, delete_request methods if needed]

def parse_supplier_data(csv_file, json_file):
    try:
        supplier_data = pd.read_csv(csv_file)
        with open(json_file, 'r') as file:
            brands_dict = json.load(file)

        filtered_data = []
        for _, row in supplier_data.iterrows():
            brand = row['Brand']
            if brand in brands_dict.values():
                filtered_data.append({
                    "SKU": row['SKU'],
                    "Name": row['Name'],
                    "Brand": brand,
                    "Cost": row['Cost']
                })

        return pd.DataFrame(filtered_data)
    except Exception as e:
        logging.error(f"Error processing supplier data: {e}")
        return pd.DataFrame()

def fetch_bigcommerce_products(api):
    # Assuming products' SKUs are stored in 'sku' field
    products = []
    page = 1
    while True:
        response = api.get_request(f"catalog/products?page={page}&limit=250")
        if not response or 'data' not in response:
            break
        products.extend(response['data'])
        if not response['meta']['pagination']['next_page']:
            break
        page += 1
    return {product['sku']: product for product in products if 'sku' in product}

def compare_and_generate_csv(supplier_data, bigcommerce_data):
    added_products = supplier_data[~supplier_data['SKU'].isin(bigcommerce_data.keys())]
    removed_products = pd.DataFrame([sku for sku in bigcommerce_data.keys() if sku not in supplier_data['SKU'].values], columns=['SKU'])

    changes_csv = 'changes.csv'
    with pd.ExcelWriter(changes_csv) as writer:
        added_products.to_excel(writer, sheet_name='Added Products', index=False)
        removed_products.to_excel(writer, sheet_name='Removed Products', index=False)

    return changes_csv

def update_bigcommerce_store(api, changes_csv):
    # This function should handle updating the store based on the changes
    # For now, it's a placeholder
    pass

def main():
    logging.info("Starting data processing...")
    api = BigCommerceAPI(client_id, access_token, store_hash)

    supplier_file_path = "supplier.csv"
    brands_json_path = "brands.json"
    supplier_data = parse_supplier_data(supplier_file_path, brands_json_path)

    if supplier_data.empty:
        logging.error("Supplier data processing failed. Exiting process.")
        return

    bigcommerce_data = fetch_bigcommerce_products(api)
    changes_csv = compare_and_generate_csv(supplier_data, bigcommerce_data)
    update_bigcommerce_store(api, changes_csv)

    logging.info(f"Data processing completed. Changes recorded in {changes_csv}.")

if __name__ == "__main__":
    main()


2023-11-14 20:38:01,215 - INFO - Starting data processing...
2023-11-14 20:38:01,216 - ERROR - Error processing supplier data: [Errno 2] No such file or directory: '/supplier.csv'
2023-11-14 20:38:01,217 - ERROR - Supplier data processing failed. Exiting process.


In [1]:
import pandas as pd
import difflib
import json
import logging

# Setting up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def match_product_name(supplier_name, threshold=0.8):
    try:
        best_match = None
        best_ratio = 0
        for store_name in products_df['StoreName']:
            if pd.isna(supplier_name) or pd.isna(store_name):
                continue
            ratio = difflib.SequenceMatcher(None, supplier_name, store_name).ratio()
            if ratio > best_ratio:
                best_ratio = ratio
                best_match = store_name
        return best_match if best_ratio >= threshold else None
    except Exception as e:
        logging.error(f"Error in match_product_name: {e}")
        return None

def contains_excluded_word(name, excluded_words):
    try:
        return any(word.lower() in name.lower() for word in excluded_words)
    except Exception as e:
        logging.error(f"Error in contains_excluded_word: {e}")
        return False

try:
    # Load the CSV and JSON files
    products_path = 'IMPORT.csv'
    suppliers_path = 'supplier.csv'
    brands_path = 'brands.json'

    # Reading the files
    products_df = pd.read_csv(products_path)
    suppliers_df = pd.read_csv(suppliers_path)

    if 'Item' in products_df.columns:
        products_df = products_df[products_df['Item'] != 'Image']
    else:
        logging.warning("'Item' column not found in products CSV.")

    products_df = products_df[['SKU', 'Name', 'Cost Price']]
    products_df.columns = ['SKU', 'StoreName', 'StoreCost']
    products_df = products_df.dropna(subset=['StoreName'])

    suppliers_df = suppliers_df[['Nombre', 'Precio']]
    suppliers_df.columns = ['Name', 'SupplierPrice']
    suppliers_df = suppliers_df.dropna(subset=['Name'])

    # Load and process brands data
    with open(brands_path, 'r') as file:
        brands_data = json.load(file)
    brand_map = {brand['abbreviation']: brand['full_name'] for brand in brands_data}
    suppliers_df['Brand'] = suppliers_df['Name'].apply(lambda x: x.split()[0] if pd.notna(x) else None)
    suppliers_df = suppliers_df[suppliers_df['Brand'].isin(brand_map.keys())]

    # Exclude products containing specific keywords
    excluded_words = ["Rebajado", "Oferta", "Promocion", "Antes", "Golpeado"]
    products_df = products_df[~products_df['StoreName'].apply(lambda x: contains_excluded_word(x, excluded_words))]
    suppliers_df = suppliers_df[~suppliers_df['Name'].apply(lambda x: contains_excluded_word(x, excluded_words))]

    # Apply fuzzy matching to find corresponding SKU in products_df for each supplier product
    suppliers_df['MatchedName'] = suppliers_df['Name'].apply(match_product_name)
    merged_df = pd.merge(suppliers_df, products_df, left_on='MatchedName', right_on='StoreName', how='left')

    # Identifying new products, deleted products, and products with price updates
    new_products = merged_df[merged_df['SKU'].isna()][['Name', 'SupplierPrice']]
    new_products['Type'] = 'New Product'

    deleted_products = products_df[~products_df['StoreName'].isin(merged_df['StoreName'])]
    deleted_products['Type'] = 'Deleted Product'

    updated_products = merged_df[~merged_df['SKU'].isna() & (merged_df['StoreCost'] != merged_df['SupplierPrice'])]
    updated_products['Type'] = 'Product Updated'

    # Preparing the final DataFrame
    final_df = pd.concat([
        new_products,
        deleted_products[['SKU', 'StoreName', 'StoreCost', 'Type']].rename(columns={'StoreName': 'Name'}),
        updated_products[['SKU', 'Name', 'StoreCost', 'SupplierPrice', 'Type']]
    ])
    final_df.reset_index(drop=True, inplace=True)

    # Save the final DataFrame to an Excel file
    final_output_path = 'updated_product_list_final.xlsx'
    final_df.to_excel(final_output_path, index=False)

    logging.info(f"Output saved to: {final_output_path}")

except Exception as e:
    logging.error(f"Error in main script: {e}")


/var/folders/yk/5nmbkvdx2bj8lf_myl2bx1fc0000gq/T/ipykernel_70046/263930454.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deleted_products['Type'] = 'Deleted Product'
/var/folders/yk/5nmbkvdx2bj8lf_myl2bx1fc0000gq/T/ipykernel_70046/263930454.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_products['Type'] = 'Product Updated'
2023-11-16 20:09:15,066 - INFO - Output saved to: updated_product_list_final.xlsx


In [25]:
import os
import json
import logging
import requests
import pandas as pd
from dotenv import load_dotenv
from requests.exceptions import HTTPError

# Configurations
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
load_dotenv()

# Constants
BASE_URL_TEMPLATE = "https://api.bigcommerce.com/stores/{store_hash}/v3/"

# BigCommerceAPI class for API interactions
class BigCommerceAPI:
    def __init__(self, client_id, access_token, store_hash):
        self.headers = {
            "X-Auth-Client": client_id,
            "X-Auth-Token": access_token,
            "Content-Type": "application/json"
        }
        self.base_url = BASE_URL_TEMPLATE.format(store_hash=store_hash)

    def get_request(self, url):
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except HTTPError as http_err:
            logging.error(f"HTTP error occurred: {http_err}, Response: {response.text}")
            return None
        except Exception as err:
            logging.error(f"An error occurred: {err}")
            return None

# Function to parse supplier data
def parse_supplier_data(csv_file, json_file):
    try:
        supplier_data = pd.read_csv(csv_file)
        with open(json_file, 'r') as file:
            brands_dict = json.load(file)

        # Data filtering logic
        filtered_data = [
            {
                "SKU": row['SKU'],
                "Name": row['Name'],
                "Brand": row['Brand'],
                "Cost": row['Cost']
            }
            for _, row in supplier_data.iterrows() if row['Brand'] in brands_dict.values()
        ]

        return pd.DataFrame(filtered_data)
    except FileNotFoundError as e:
        logging.error(f"File not found: {e}")
        return pd.DataFrame()
    except Exception as e:
        logging.error(f"Error processing supplier data: {e}")
        return pd.DataFrame()

# Function to fetch BigCommerce products
def fetch_bigcommerce_products(api):
    products = []
    page = 1
    while True:
        response = api.get_request(f"{api.base_url}catalog/products?page={page}&limit=250&include=variants")
        if response is None or 'data' not in response:
            break

        logging.info(f"Fetched Page {page}: {len(response['data'])} products")
        products.extend(response['data'])

        # Pagination logic
        next_page = response.get('meta', {}).get('pagination', {}).get('next_page')
        if not next_page:
            break
        page += 1

    return products

def fetch_and_process_products(api):
    products = []
    page = 1
    while True:
        response = api.get_request(f"{api.base_url}catalog/products?page={page}&limit=250&include=variants")
        if response is None or 'data' not in response:
            break

        logging.info(f"Fetched Page {page}: {len(response['data'])} products")
        products.extend(response['data'])
        next_page = response.get('meta', {}).get('pagination', {}).get('next_page')
        if not next_page:
            break
        page += 1

    all_product_frames = []
    for product in products:
        if 'variants' in product:
            product_variants_df = pd.json_normalize(
                product,
                record_path='variants',
                meta=[
                    'id', 
                    'name', 
                    'price', 
                    'cost_price',
                    ['custom_url', 'url']
                ],
                meta_prefix='product_',  # Prefix for product-level fields
                record_prefix='variant_',  # Prefix for variant-level fields
                errors='ignore'
            )

            # Rename columns to match your desired DataFrame structure
            product_variants_df.rename(columns={
                'product_id': 'ProductID',
                'product_name': 'ProductName',
                'variant_sku': 'SKU',
                'product_custom_url.url': 'URL',
                'product_price': 'ListPrice',
                'product_cost_price': 'StoreCost'
            }, inplace=True)

            all_product_frames.append(product_variants_df)

    if all_product_frames:
        combined_df = pd.concat(all_product_frames, ignore_index=True)
        return combined_df

    return pd.DataFrame()



# Main function
def main():
    try:
        # API initialization
        client_id = os.getenv("BIGCOMMERCE_CLIENT_ID")
        access_token = os.getenv("BIGCOMMERCE_ACCESS_TOKEN")
        store_hash = os.getenv("BIGCOMMERCE_STORE_HASH")
        api = BigCommerceAPI(client_id, access_token, store_hash)

        # Fetching and processing products
        products_data = fetch_bigcommerce_products(api)
        if not products_data:
            logging.error("No products data fetched from BigCommerce")
            return

        # Processing fetched products data
        products_df = fetch_and_process_products(api)
        if products_df.empty:
            logging.error("Processed products data is empty.")
            return

        # Ensure that the 'ProductName' column exists before applying the filter
        if 'ProductName' not in products_df.columns:
            logging.error("'ProductName' column not found in products data")
            return

        # Exclude products containing specific keywords
        excluded_words = ["Rebajado", "Oferta", "Promocion", "Antes", "Golpeado"]
        products_df = products_df[~products_df['ProductName'].apply(lambda x: contains_excluded_word(x, excluded_words))]
        # Parse supplier data
        supplier_csv = 'supplier.csv'
        brands_json = 'brands.json'
        suppliers_df = parse_supplier_data(supplier_csv, brands_json)

        if suppliers_df.empty:
            logging.error("Supplier data is empty or could not be processed.")
            return
        # Apply fuzzy matching to find corresponding SKU in products_df for each supplier product
        suppliers_df['MatchedName'] = suppliers_df['Name'].apply(match_product_name)
        merged_df = pd.merge(suppliers_df, products_df, left_on='MatchedName', right_on='Name', how='left')

        # Identifying new products, deleted products, and products with price updates
        new_products = merged_df[merged_df['SKU'].isna()][['Name', 'SupplierPrice']]
        new_products['Type'] = 'New Product'

        # Fix for 'SettingWithCopyWarning'
        deleted_products = products_df.loc[~products_df['ProductName'].isin(merged_df['ProductName']), :].copy()
        deleted_products['Type'] = 'Deleted Product'

        updated_products = merged_df.loc[~merged_df['SKU'].isna() & (merged_df['StoreCost'] != merged_df['SupplierPrice']), :].copy()
        updated_products['Type'] = 'Product Updated'
        # Preparing the final DataFrame
        final_df = pd.concat([
            new_products,
            deleted_products[['SKU', 'Name', 'StoreCost', 'Type']],
            updated_products[['SKU', 'Name', 'StoreCost', 'SupplierPrice', 'Type']]
        ])
        final_df.reset_index(drop=True, inplace=True)

       # Save the final DataFrame to an Excel file
        final_output_path = 'updated_product_list_final.xlsx'
        final_df.to_excel(final_output_path, index=False)
        logging.info(f"Output saved to: {final_output_path}")

    except Exception as e:
        logging.error(f"An error occurred in main: {e}")

if __name__ == "__main__":
    main()

2023-11-16 21:11:32,930 - INFO - Fetched Page 1: 250 products
2023-11-16 21:11:35,274 - INFO - Fetched Page 1: 250 products
2023-11-16 21:11:35,582 - ERROR - Error processing supplier data: 'Brand'
2023-11-16 21:11:35,583 - ERROR - Supplier data is empty or could not be processed.
